<a href="https://colab.research.google.com/github/VirenderTanwar/Machine-Learning-with-Python/blob/master/Text_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
!wget https://www.dropbox.com/s/kfye28it99g9j12/imdb_labelled.txt

--2020-02-06 19:03:15--  https://www.dropbox.com/s/kfye28it99g9j12/imdb_labelled.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kfye28it99g9j12/imdb_labelled.txt [following]
--2020-02-06 19:03:15--  https://www.dropbox.com/s/raw/kfye28it99g9j12/imdb_labelled.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbd0efa899fc4eebea55eb6e60c.dl.dropboxusercontent.com/cd/0/inline/AxkbspFVl-Ckb-p29LhaQDx1KfJmz5l6ZiYXofD0Xd41HtaG5j64WpNv9vGXB8YwrKXvk55lswIqFiJGAkocGWsWpKaEKPF8QkBksGYUhfsjuA/file# [following]
--2020-02-06 19:03:16--  https://ucbd0efa899fc4eebea55eb6e60c.dl.dropboxusercontent.com/cd/0/inline/AxkbspFVl-Ckb-p29LhaQDx1KfJmz5l6ZiYXofD0Xd41HtaG5j64WpNv9vGXB8YwrKXvk55lswIqFiJGAkocGWsWpKaEKPF8QkBksGYUhfsjuA/file
R

In [0]:
file=r'/content/imdb_labelled.txt'

In [0]:
import pandas as pd
imbd=pd.read_csv(file,sep='\t',header=None,names=['review','target'])

In [0]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-02-06 19:14:07--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-02-06 19:14:07--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-02-06 19:14:08--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [0]:
!rm /content/AAAw1_JIzpuVvwteJCma0xMla?dl=0.1

In [0]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
import numpy as np

In [0]:
embedding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
  values=line.split()
  word=values[0]
  coefs=np.asarray(values[1:],dtype='float32')
  embedding_index[word]=coefs
f.close()

In [0]:
from sklearn.model_selection import train_test_split
data_train,data_test=train_test_split(imbd,test_size=0.2)

In [0]:
x_train=data_train['review']
y_train=data_train['target']
x_test=data_test['review']
y_test=data_test['target']

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation,Dense,Input,Embedding,Dropout
from tensorflow.keras.models import Model

In [0]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
max(sent_lens)

1616

In [0]:
np.percentile(sent_lens,95)

41.0

In [0]:
max_len=40

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [0]:
vocab_size

2833

In [0]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector


In [0]:
inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer=LSTM(50)(embed)
dense1=Dense(10,activation='relu')(lstm_layer)
drop=Dropout(0.2)(dense1)
final_layer=Dense(1,activation='sigmoid')(drop)

model=Model(inputs=inputs,outputs=final_layer)

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 40)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 40, 300)           850200    
_________________________________________________________________
lstm (LSTM)                  (None, 50)                70200     
_________________________________________________________________
dense (Dense)                (None, 10)                510       
_________________________________________________________________
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 920,921
Trainable params: 70,721
Non-trainable params: 850,200
__________________________________________________

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(seq_train_matrix,y_train.values,validation_data=[seq_test_matrix,y_test.values],epochs=40,class_weight={0:1,1:12},
          batch_size=50)
p=model.predict(seq_test_matrix)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 598 samples, validate on 150 samples
Epoch 1/40
598/598 [==============================] - 4s 6ms/sample - loss: 4.0906 - accuracy: 0.5117 - val_loss: 2.8570 - val_accuracy: 0.4733
Epoch 2/40
598/598 [==============================] - 1s 1ms/sample - loss: 2.7595 - accuracy: 0.5301 - val_loss: 2.0175 - val_accuracy: 0.4667
Epoch 3/40
598/598 [==============================] - 1s 861us/sample - loss: 2.0920 - accuracy: 0.5284 - val_loss: 1.7811 - val_accuracy: 0.4667
Epoch 4/40
598/598 [==============================] - 1s 968us/sample - loss: 1.9101 - accuracy: 0.5284 - val_loss: 1.7631 - val_accuracy: 0.4667
Epoch 5/40
598/598 [==============================] - 1s 972us/sample - loss: 1.9194 - accuracy: 0.5268 - val_loss: 1.7594 - val_accuracy: 0.4667
Epoch 6/40
598/598 [==============================] - 1s 921us/sample - loss: 1.9195 - accuracy: 0.5284 - val_loss: 1.7377 - val_accuracy: 0.4667
Epoch 7/40
598/598 [============

In [0]:
p

array([[0.9998516 ],
       [0.9994993 ],
       [0.00579112],
       [0.99968433],
       [0.9998845 ],
       [0.010332  ],
       [0.00870566],
       [0.98379135],
       [0.9929807 ],
       [0.9991211 ],
       [0.99973637],
       [0.9998776 ],
       [0.4163789 ],
       [0.9982628 ],
       [0.92832816],
       [0.9978822 ],
       [0.9953231 ],
       [0.99976677],
       [0.00340438],
       [0.9998466 ],
       [0.9977481 ],
       [0.9998846 ],
       [0.9999044 ],
       [0.7448833 ],
       [0.8739525 ],
       [0.43967196],
       [0.08670631],
       [0.75634813],
       [0.00937072],
       [0.1647998 ],
       [0.9609995 ],
       [0.999824  ],
       [0.08456764],
       [0.10473842],
       [0.00431661],
       [0.99980766],
       [0.99990773],
       [0.04995671],
       [0.19084229],
       [0.9948197 ],
       [0.94573456],
       [0.00453813],
       [0.059568  ],
       [0.96348363],
       [0.02271755],
       [0.02231664],
       [0.99735105],
       [0.006

In [0]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,p)

0.88875